# Generate test data simulating calls in an emergency dispatch center

### Install NuGet-Packages +  Using-Statements

In [1]:
#r "nuget:MathNet.Numerics"
#r "nuget:CsvHelper"

Installing package MathNet.Numerics.......................................done!

Successfully added reference to package MathNet.Numerics, version

Installing package CsvHelper........done!

Successfully added reference to package CsvHelper, version

In [2]:
using MathNet.Numerics.Distributions;
using XPlot.Plotly;

### Model class

In [3]:
class Call 
{
    public DateTime Date { get; set; }
    
    public int Month { get; set; }
    
    public string DayOfWeek { get; set; }
        
    public int Hour { get; set; }
    
    public string WeatherConditions { get; set; }
}

### Generate random data for one day

In [15]:
var random = new Random();

private IEnumerable<Call> GenerateDay(DateTime date)
{
    double callCount = 500+random.Next(100);
    
    // up to 20 % more calls between November and February
    if (date.Month >= 11 || date.Month <= 2)
    {
        callCount *= 1 + random.NextDouble() * 0.2;
    }
    
    string weatherConditions = random.NextDouble() > 0.7 ? "bad" : "good";
    
    // 40-60 % more calls when bad weather conditions occur
    if (weatherConditions == "bad")
    {
        callCount *= 1.4 + random.NextDouble() * 0.2;
    }
    
    var weekDayFactor = new Dictionary<DayOfWeek, float>()
    {
        { DayOfWeek.Monday, 1.4f },
        { DayOfWeek.Tuesday, 1.2f },
        { DayOfWeek.Wednesday, 1.0f },
        { DayOfWeek.Thursday, 1.2f },
        { DayOfWeek.Friday, 1.4f },
        { DayOfWeek.Saturday, 0.8f },
        { DayOfWeek.Sunday, 0.3f },
    };
    
    // consider day of week factor
    callCount *= weekDayFactor[date.DayOfWeek];
    
    Normal normalDist = new Normal(11, 3);
    var values = normalDist.Samples().Take((int)Math.Floor(callCount)).ToList();

    var calls = values.Select(v => new Call()
    {
        Date = date.Date,
        Month = date.Month,
        DayOfWeek = date.DayOfWeek.ToString(),
        Hour = (int)Math.Round(v),
        WeatherConditions = weatherConditions
    }).ToList();
    
    return calls;
}


### Generate one year

In [16]:
DateTime currentDate = new DateTime(2019, 1, 1);
var calls = new List<Call>();

for (int i=0; i<365; i++) 
{
   calls.AddRange(GenerateDay(currentDate));
   currentDate = currentDate.AddDays(1);
}

calls.Count

247854

### Visualize generated data

In [17]:
using System.Linq;

var callsByDate = calls.GroupBy(c => c.Date).ToDictionary(c => c.Key, c => c.Count());

Chart.Plot(
    new Graph.Scatter() {
        x = callsByDate.Keys,
        y = callsByDate.Values
    }
)

### Group generated calls by hour

In [18]:
using System.Linq;

var groupedByHour = (from c in calls
                     group c by new
                     {
                         c.Date,
                         c.Hour,
                         c.WeatherConditions
                     }
                     into grouped
                     orderby grouped.Key.Date, grouped.Key.Hour
                     select new
                     {
                         Date = grouped.Key.Date,
                         Hour = grouped.Key.Hour,
                         Month = grouped.Key.Date.Month,
                         DayOfWeek = grouped.Key.Date.DayOfWeek.ToString(),
                         WeatherConditions = grouped.Key.WeatherConditions,
                         Calls = grouped.Count()
                     }).ToList();

groupedByHour.Take(100)

index,Date,Hour,Month,DayOfWeek,WeatherConditions,Calls
0,2019-01-01 00:00:00Z,1,1,Tuesday,good,1
1,2019-01-01 00:00:00Z,2,1,Tuesday,good,1
2,2019-01-01 00:00:00Z,3,1,Tuesday,good,7
3,2019-01-01 00:00:00Z,4,1,Tuesday,good,4
4,2019-01-01 00:00:00Z,5,1,Tuesday,good,18
5,2019-01-01 00:00:00Z,6,1,Tuesday,good,23
6,2019-01-01 00:00:00Z,7,1,Tuesday,good,41
7,2019-01-01 00:00:00Z,8,1,Tuesday,good,68
8,2019-01-01 00:00:00Z,9,1,Tuesday,good,84
9,2019-01-01 00:00:00Z,10,1,Tuesday,good,82


### Export to CSV

In [20]:
using CsvHelper;

using (var writer = new System.IO.StreamWriter("data\\calls.csv"))
using (var csv = new CsvWriter(writer, System.Globalization.CultureInfo.InvariantCulture))
{    
    csv.WriteRecords(groupedByHour);
}

### Add columns with data from previous hours

In [21]:
var addPrevious = (from g in groupedByHour
                  let prev = groupedByHour.FirstOrDefault(p => p.Date == g.Date && p.Hour == g.Hour - 1)
                  let prev2 = groupedByHour.FirstOrDefault(p => p.Date == g.Date && p.Hour == g.Hour - 2)
                  select new
                  {
                      g.Date,
                      g.Hour,
                      g.Month,
                      g.DayOfWeek,
                      g.WeatherConditions,
                      g.Calls,
                      CallsPrevHour = prev?.Calls ?? 0,
                      CallsPrevHour2 = prev2?.Calls ?? 0
                  }).ToList();
  


### Export to CSV

In [22]:
using CsvHelper;

using (var writer = new System.IO.StreamWriter("data\\calls-previous.csv"))
using (var csv = new CsvWriter(writer, System.Globalization.CultureInfo.InvariantCulture))
{    
    csv.WriteRecords(addPrevious);
}